## Presented in WavyTropics Workshop on Fri 24 Nov 2023

### Author : Donaldi Permana
### ORCID : https://orcid.org/0000-0001-5674-9238
### Affiliation : R&D Center of BMKG
### Email : donaldi.permana@bmkg.go.id

Disclaimer: This script is for educational purposes only and is intended as an exercise toward more understanding about the topic.

In [ ]:
# Created on Fri Nov 10 15:53:00 2023
#@author: donaldi permana

In [ ]:
# install necessary modules
#!pip install cartopy
#!pip install xarray netcdf4 pydap h5pyd

In [ ]:
import xarray as xr
import cartopy.crs as ccrs
import cartopy as cart
import numpy as np
import warnings
import pandas as pd
import os
import datetime
%matplotlib inline
import matplotlib.pyplot as pl
warnings.filterwarnings("ignore")
warnings.simplefilter('ignore')

#### Source : https://resources.eumetrain.org/satmanu/CM4SH/InNCS/index.htm

### Cold Surge Life cycle
An NCS event may last from a few days to a week.

#### Phase 0: Siberia - Pressure surge outbreaks

The characteristics of cold air outbreaks are closely associated with the formation, intensification, 
southward movement, transformation and dissipation of the Siberian-Mongolian High. 
The surge initiates when the Siberian high intensifies above 1045 hPa.

#### Case study : period of 20 Jan - 10 Feb 2007

In [ ]:
# Case study
year_start = 2007
year_end = 2007
time_bnd = [str(year_start)+'-01-20', str(year_end)+'-02-10']

smh_lat_bnd = [37, 55]
smh_lon_bnd = [80, 110]

var = 'mslp' # resolution 2.5 deg
url = 'http://apdrc.soest.hawaii.edu:80/dods/public_data/Reanalysis_Data/NCEP/NCEP2/daily/surface/mslp'
#print('Reading mean sea-level pressure data from '+ url)
ds = xr.open_dataset(url)
# subsetting domain
ds = ds.sel(lat=slice(*smh_lat_bnd),lon=slice(*smh_lon_bnd),time=slice(*time_bnd))
ds1 = ds.quantile(0.99, dim=(['lat','lon']))/100
ds2 = ds.mean(dim=(['lat','lon']))/100
pl.rcParams.update({'font.size': 12})
fig, ax = pl.subplots(1, figsize=(15, 3))
ds1[var].plot(ax = ax,label='p99',marker='o')
ds2[var].plot(ax = ax,label='mean',color='k',linestyle='--',marker='o')
ax.axhline(y=1045, color='r', linestyle='--', label='threshold')
ax.legend()
ax.set_title('Siberian - Mongolian High. MSLP')

# save figure
fig.savefig('SMH_MSLP.png', format='png', dpi=150, bbox_inches='tight')

#### Phase 1: The arrival of the surge

The NCS's arrival at Hong Kong is marked by a dramatic 12 - 24 hour change in synoptic conditions. 
Wu and Chan (1995) showed that the NCS brings decreased near-surface temperature (around 4-5 °C), 
decreased humidity (shown by an increase in dew point depression around 1-2 °C), 
increased surface pressures (approximately +3 hPa), and strong northerly winds of around 5.5 m/s within a period of 12 - 24 hours. 
Riehl (1968) proposed a large pressure difference between 30°N, 115°E and Hong Kong exceeding 10 mb as the surge forecast criterion of an NCS.

In [ ]:
var = 'mslp' # resolution 2.5 deg
url = 'http://apdrc.soest.hawaii.edu:80/dods/public_data/Reanalysis_Data/NCEP/NCEP2/daily/surface/mslp'
#print('Reading mean sea-level pressure data from '+ url)
ds = xr.open_dataset(url)

ds1 = ds.sel(lat=30,lon=115,method='nearest').sel(time=slice(*time_bnd))/100 # MSLP at Gushi
ds2 = ds.sel(lat=20,lon=115,method='nearest').sel(time=slice(*time_bnd))/100 # MSLP at HK
ds3 = ds1-ds2

pl.rcParams.update({'font.size': 12})
fig, (ax1, ax2) = pl.subplots(2, figsize=(15, 6))
ds3[var].plot(ax = ax1,label='MSLP Diff Gushi - HK',marker='o')
ax1.legend()
ax1.set_xticklabels('')
ax1.set_xlabel('')
ax1.axhline(y=10, color='r', linestyle='--', label='threshold')
ax1.set_title('MSLP Diff Gushi - HK')

var = 't2' # resolution 0.25 deg
url = 'http://apdrc.soest.hawaii.edu:80/dods/public_data/Reanalysis_Data/ERA5/hourly/2m_temperature'
ds = xr.open_dataset(url)
#HK_lat_bnd = [22, 22.5]
#HK_lon_bnd = [114, 114.5]
ds1 = ds.sel(lat=22.3193,lon=114.1694, method='nearest').sel(time=slice(*time_bnd))-273.15 # Temp at HK
ds2 = ds1#.resample(time='D').mean()#.mean(dim=(['lat','lon']))
time_shift = 12
ds3 = ds2.shift(time=time_shift)
ds4 = ds3-ds2
ax4 = ax2.twinx()
ds2[var].plot(ax=ax2,label='Temp at HK')
ds4[var].plot(ax=ax4,label=str(time_shift)+' hr Temp. Diff',color='k',linestyle='--')
ax4.axhline(y=-4, color='r', linestyle='--', label='threshold')
ax2.legend()
ax4.legend()
ax2.set_title('')
ax4.set_title(str(time_shift)+' hr Temp. Diff')

# save figure
fig.savefig('GushiHK.png', format='png', dpi=150, bbox_inches='tight')

#### Phase 2: NCS propagation through South China Sea

On synoptic time scales, northeasterly cold surges often dominate the low-level circulation patterns over the equatorial South China Sea (SCS). 
Although cold surge winds are typically dry, they gain moisture while crossing bodies of warm water, 
thus enhancing the East Asian Hadley Cell together with the strengthening upper-tropospheric flow.

Chang et. al. (2005) devised a cold surge index to define the occurrence of cold surge in this area using 
the average 925 hPa meridional wind between 110°E and 117.5°E along 15°N. 
A cold surge event occurs when this index exceeds 8 m/s. 
Furthermore, based on the magnitude of the meridional wind, the surge is divided into the following categories:
##### Weak surge: surge index between 8 and 10 m/s
##### Moderate surge: surge index between 10 and 12 m/s
##### Strong surge: surge index greater than 12 m/s

In [ ]:
lon_bnd = [110, 117.5]
lat = 15
level = 925

var = 'v' # resolution 0.25 deg
url = 'http://apdrc.soest.hawaii.edu:80/dods/public_data/Reanalysis_Data/ERA5/daily_3d/V_component_of_wind'
#print('Reading mean sea-level pressure data from '+ url)
ds = xr.open_dataset(url)
# subsetting domain
ds = ds.sel(lat=lat,method='nearest').sel(lon=slice(*lon_bnd),time=slice(*time_bnd),lev=level)
ds1 = ds.mean(dim=(['lon']))
pl.rcParams.update({'font.size': 12})
fig, ax = pl.subplots(1, figsize=(15, 3))
ds1[var].plot(ax = ax,label='meridional wind 15N', marker='o')
ax.axhline(y=-8, color='g', linestyle='--',label='weak surge')
ax.axhline(y=-10, color='orange', linestyle='--', label='moderate surge')
ax.axhline(y=-12, color='r', linestyle='--', label='strong surge')
ax.legend()
ax.set_title('Average Meridional Wind at 15°N, 110-117.5°E')

# save figure
fig.savefig('WIND15N.png', format='png', dpi=150, bbox_inches='tight')

#### Phase 3: Cross-Equatorial Northerly Surge (CENS)

The cold surge can transport a low-level air mass from the mid-latitudes to the equator quite effectively. 
As this cold and dry air mass (referred to as 'cold tongue') moves into low latitudes, 
strong surface heat fluxes weaken the cold air anomalies, and the surge may lose its 'cold' character. 
However, strong meridional winds remain as clear signatures of the surge. 
This factor is important in for the formation of precipitation over the Maritime Continent. 
Hattori (2011) defined the Cross-Equatorial Northerly Surge (CENS) index as the speed of the average meridional wind 
that exceeds 5 m/s in the area (over 105°E - 115°E, 5°S - EQ).

In [ ]:
lat_bnd = [-5, 0]
lon_bnd = [105, 115]
level = 975

var = 'v' # resolution 0.25 deg
url = 'http://apdrc.soest.hawaii.edu:80/dods/public_data/Reanalysis_Data/ERA5/daily_3d/V_component_of_wind'
#print('Reading mean sea-level pressure data from '+ url)
ds = xr.open_dataset(url)
# subsetting domain
ds = ds.sel(lat=slice(*lat_bnd),lon=slice(*lon_bnd),time=slice(*time_bnd),lev=level)
ds1 = ds.mean(dim=(['lon','lat']))
pl.rcParams.update({'font.size': 12})
fig, ax = pl.subplots(1, figsize=(15, 3))
ds1[var].plot(ax = ax,label='CENS index',marker='o')
ax.axhline(y=-5, color='r', linestyle='--', label='threshold')
ax.legend()
ax.set_title('CENS index')

# save figure
fig.savefig('CENS.png', format='png', dpi=150, bbox_inches='tight')

### Interaction CS and MJO

References:

Lim, S. Y., Marzin, C., Xavier, P., Chang, C. P., & Timbal, B. (2017).
Impacts of boreal winter monsoon cold surges and the interaction with MJO on Southeast Asia rainfall.
Journal of Climate, 30(11), 4267-4281.

The CS day is defined when northeasterly wind averaged over 
the SCS (5°–10°N,107°–115°E) at 850 hPa exceeds a threshold
equalling 0.75 times the standard deviation above the long-term mean 
(northeasterly windspeed exceeding 9.07 m⋅s−1) and
averaged MSLP over (105°–122°E, 18°–22°N) exceeding 1,020 hPa
for at least two consecutive days or if there is a break in northeasterly wind speed 
(when the northeasterly winds speed fall back to less than 9.07 m⋅s−1) of not more than two consecutive days.
Other criteria : Calm to easterly wind (u <= 0 m/s) ; northerly wind (v < 0 m/s)

Daily datasets (precipitation, 850-hPa winds, MSLP, and MJO index) for November, December, January, February and March (NDJFM) from 1998 to 2020 covering the SEA region.

The daily MSLP and 850-hPa winds are obtained from NCEP2 at 2.5x2.5 deg and ERA5 at 0.25x0.25 deg resolutions, respectively
The daily mean precipitation data are obtained from the CMORPH V1.0 dataset with a resolution of 0.25x0.25 deg.
The MJO index

In [ ]:
# Domain D1
d1_min_lon = 107
d1_max_lon = 115
d1_min_lat = 5
d1_max_lat = 10
d1_lat_bnd = [d1_min_lat, d1_max_lat]
d1_lon_bnd = [d1_min_lon, d1_max_lon]

# Domain D2
d2_min_lon = 105
d2_max_lon = 122
d2_min_lat = 18
d2_max_lat = 22
d2_lat_bnd = [d2_min_lat, d2_max_lat]
d2_lon_bnd = [d2_min_lon, d2_max_lon]

year_start = 1998
year_end = 2020
time_bnd = [str(year_start)+'-01-01', str(year_end)+'-12-31'] # ?? years

level = '850' # mb

#print(level)

In [ ]:
import math

## define wind conversion function

def windcalc(u,v):
    ws = np.sqrt(u*u + v*v)    
    wd = np.mod(180+np.rad2deg(np.arctan2(u, v)),360)
    return ws,wd
    
def uvcalc(ws,wd):
    wd = 270-wd
    u = np.empty(wd.shape)
    v = np.empty(wd.shape)
    for i in range(wd.shape[0]):
        for j in range(wd.shape[1]):
            u[i,j] = ws[i,j] * math.cos(math.radians(wd[i,j]))
            v[i,j] = ws[i,j] * math.sin(math.radians(wd[i,j]))
    return u,v

In [ ]:
## Get the datasets

## Zonal wind from ERA5 Dataset (0.25 deg)
var_u = 'u'
filename = var_u+level+'_'+str(year_start)+'_'+str(year_end)+'.nc'
if not os.path.isfile(filename):
  url_u = 'http://apdrc.soest.hawaii.edu:80/dods/public_data/Reanalysis_Data/ERA5/daily_3d/U_component_of_wind'
  print('Reading ERA5 data from '+ url_u)
  dsu = xr.open_dataset(url_u)
  # subsetting domain D1
  dsu_d1 = dsu.sel(lat=slice(*d1_lat_bnd),lon=slice(*d1_lon_bnd),time=slice(*time_bnd),lev=level)
  ## save to netcdf
  dsu_d1.to_netcdf(var_u+level+'_'+str(year_start)+'_'+str(year_end)+'.nc')
else:
  pl.rcParams.update({'font.size': 12})
  fig, ax = pl.subplots(1, figsize=(15, 3))
  print('Reading '+var_u+' from '+ filename)
  dsu = xr.open_mfdataset(var_u+level+'_'+str(year_start)+'_'+str(year_end)+'.nc')
  # subsetting domain D1
  dsu_d1 = dsu.sel(lat=slice(*d1_lat_bnd),lon=slice(*d1_lon_bnd),time=slice(*time_bnd))
  #print(dsu_d1)
  #dsu_d1[var_u].isel(time=100).plot.contourf()
  # average in D1
  dsu_d1_mean = dsu_d1.mean(dim=(['lat','lon']))
  #print(dsu_d1_mean)
  dsu_d1_mean[var_u].plot()
  # Get only NDJFM months
  dsu_d1_mean_ndjfm = dsu_d1_mean.where(((dsu_d1_mean['time.month'].isin([11,12,1,2,3]))), drop=True)
  dsu_d1_mean_ndjfm[var_u].plot()


In [ ]:
## Meridional wind from ERA5 Dataset (0.25 deg)
var_v = 'v'
filename = var_v+level+'_'+str(year_start)+'_'+str(year_end)+'.nc'
if not os.path.isfile(filename):
  url_v = 'http://apdrc.soest.hawaii.edu:80/dods/public_data/Reanalysis_Data/ERA5/daily_3d/V_component_of_wind'
  print('Reading ERA5 data from '+ url_v)
  dsv = xr.open_dataset(url_v)
  # subsetting domain D1
  dsv_d1 = dsv.sel(lat=slice(*d1_lat_bnd),lon=slice(*d1_lon_bnd),time=slice(*time_bnd),lev=level)
  ## save to netcdf
  dsv_d1.to_netcdf(var_v+level+'_'+str(year_start)+'_'+str(year_end)+'.nc')
else:
  pl.rcParams.update({'font.size': 12})
  fig, ax = pl.subplots(1, figsize=(15, 3))
  print('Reading '+var_v+' from '+ filename)
  dsv = xr.open_mfdataset(var_v+level+'_'+str(year_start)+'_'+str(year_end)+'.nc')
  # subsetting domain D1
  dsv_d1 = dsv.sel(lat=slice(*d1_lat_bnd),lon=slice(*d1_lon_bnd),time=slice(*time_bnd))
  #print(dsv_d1)
  #dsv_d1[var_v].isel(time=100).plot.contourf()
  # average in D1
  dsv_d1_mean = dsv_d1.mean(dim=(['lat','lon']))
  #print(dsv_d1_mean)
  dsv_d1_mean[var_v].plot()
  # Get only NDJFM months
  dsv_d1_mean_ndjfm = dsv_d1_mean.where(((dsv_d1_mean['time.month'].isin([11,12,1,2,3]))), drop=True)
  dsv_d1_mean_ndjfm[var_v].plot()

In [ ]:
## MSLP from NCEP2 Dataset (resolution 2.5 deg)
var_sp = 'mslp' 
filename = var_sp+level+'_'+str(year_start)+'_'+str(year_end)+'.nc'
if not os.path.isfile(filename):
  url_sp = 'http://apdrc.soest.hawaii.edu:80/dods/public_data/Reanalysis_Data/NCEP/NCEP2/daily/surface/mslp'
  print('Reading NCEP2 data from '+ url_sp)
  dssp = xr.open_dataset(url_sp)
  # subsetting domain D2
  dssp_d2 = dssp.sel(lat=slice(*d2_lat_bnd),lon=slice(*d2_lon_bnd),time=slice(*time_bnd))/100
  ## converting hourly to daily data
  #dssp_d2 = dssp_d2.resample(time='D').pad() # for the ease, we use pad() instead of mean()
  ## save to netcdf
  dssp_d2.to_netcdf(var_sp+level+'_'+str(year_start)+'_'+str(year_end)+'.nc')
else:
  pl.rcParams.update({'font.size': 12})
  fig, ax = pl.subplots(1, figsize=(15, 3))
  print('Reading '+var_sp+' from '+ filename)
  dssp = xr.open_mfdataset(var_sp+level+'_'+str(year_start)+'_'+str(year_end)+'.nc')
  # subsetting domain D2
  dssp_d2 = dssp.sel(lat=slice(*d2_lat_bnd),lon=slice(*d2_lon_bnd),time=slice(*time_bnd))
  #print(dssp_d2)
  #dssp_d2[var_sp].isel(time=100).plot.contourf()
  # max in D2
  dssp_d2_mean = dssp_d2.max(dim=(['lat','lon']))
  #print(dssp_d2_mean)
  dssp_d2_mean[var_sp].plot()
  # Get only NDJFM months
  dssp_d2_mean_ndjfm = dssp_d2_mean.where(((dssp_d2_mean['time.month'].isin([11,12,1,2,3]))), drop=True)
  dssp_d2_mean_ndjfm[var_sp].plot()

In [ ]:
pl.rcParams.update({'font.size': 12})
fig, ax = pl.subplots(1, figsize=(15, 3))

# calculate wind speed and wind direction
[ws,wd] = windcalc(dsu_d1_mean_ndjfm[var_u].values, dsv_d1_mean_ndjfm[var_v].values)
#print(ws)
normalized_ws = (ws - np.mean(ws))/np.std(ws)
dsws_d1_mean_ndjfm = dsu_d1_mean_ndjfm.copy()
dsws_d1_mean_ndjfm['ws'] = dsws_d1_mean_ndjfm[var_u]
dsws_d1_mean_ndjfm['nws'] = dsws_d1_mean_ndjfm[var_u]
dsws_d1_mean_ndjfm['ws'].values = ws
dsws_d1_mean_ndjfm['nws'].values = normalized_ws
dsws_d1_mean_ndjfm['ws'].plot()
dsws_d1_mean_ndjfm['nws'].plot()

In [ ]:
# Get CS days based on criteria

cond1 = dsu_d1_mean_ndjfm[var_u].values <= 0 # Calm to easterly wind (u <= 0 m/s)
cond2 = dsv_d1_mean_ndjfm[var_v].values < 0 # northerly wind (v < 0 m/s)
cond3 = dsws_d1_mean_ndjfm['nws'].values >= 0.75 # northeasterly wind averaged over the SCS (5◦–10◦N,107◦–115◦E) 
# at 850 hPa exceeds a threshold equalling 0.75 times the standard deviation above the long-term mean 
cond4 = dssp_d2_mean_ndjfm[var_sp].values >= 1020 # averaged MSLP over (105–122◦E, 18–22◦N) exceeding 1,020 hPa 

idx_CS = np.where(cond1 & cond2 & cond3 & cond4)[0]
all_ndjfm_days = dsu_d1_mean_ndjfm[var_u].time
total_days_ndjfm = len(dsu_d1_mean_ndjfm[var_u].values)
#print(total_days_ndjfm)
print('During NDJFM from '+str(year_start)+ ' to '+ str(year_end)+ 
      ', there are '+ str(len(idx_CS)) +' CS days') #(' +str(100*len(idx_CS)/total_days_ndjfm)+' %)')
days_CS = dsu_d1_mean_ndjfm.isel(time=idx_CS).time
days_CS_bymonth = days_CS.groupby('time.month').count('time')
print('Months : ' + str(days_CS_bymonth.month.values))
print('CS Days : ' + str(days_CS_bymonth.values))
pl.rcParams.update({'font.size': 12})
fig, ax = pl.subplots(1, figsize=(5, 3))
ax.bar(days_CS_bymonth.month, days_CS_bymonth.values)
ax.set_title('CS Days')

# number of CS days without MSLP criteria
idx_CS_nMSLP = np.where((cond1 & cond2 & cond3))[0]
#print(len(idx_CS_nMSLP))
days_CS_nMSLP = dsu_d1_mean_ndjfm.isel(time=idx_CS_nMSLP).time

# number of no CS days
idx_nCS = np.where(np.bitwise_not(cond1 & cond2 & cond3 & cond4))[0]
#print(len(idx_nCS))
days_nCS = dsu_d1_mean_ndjfm.isel(time=idx_nCS).time

In [ ]:
# Read MJO index and retrieve index for NDJFM months
mjo_filename = 'mjo2.csv'
dff = pd.read_csv(mjo_filename)
dff['DATETIME'] = pd.to_datetime(dff['year'].astype(str)+'-'+dff['month'].astype(str)+'-'+dff['day'].astype(str), format='%Y-%m-%d')

init_year = int(year_start)
end_year = int(year_end)

year = dff['year'].astype(int)
month = dff['month']
day = dff['day']
RMM1 = dff['RMM1']
RMM2 = dff['RMM2']
phase = dff['phase'].astype(float)
amplitude = dff['amplitude']

#mjo_idx = np.where((year>=init_year) & (year<=end_year))[0]
# remove feb 29 from leap year for simplification
#leap_idx = np.where((np.mod(year[mjo_idx],4) == 0) & (month[mjo_idx] == 2) & (day[mjo_idx] == 29))[0]
#mjo_idx = np.delete(mjo_idx, leap_idx)

In [ ]:
# Get MJO days based on criteria

# MJO - All active MJO in phases 2–4 [amplitude >= 1; based on MJO index defined by Wheeler and Hendon (2004)].
# nMJO - All days with no active MJO in phases 2–4 (this includes MJO days in phases 1 and 5–8).

cond_year = (year>=init_year) & (year<=end_year)
cond_ndjfm = ((month>=1) & (month<=3)) | ((month>=11) & (month<=12))
cond_phase24 = ((phase>=2) & (phase<=4))
cond_amplitude1 = (amplitude>=1)

idx_MJO = np.where(cond_year & cond_ndjfm & cond_phase24 & cond_amplitude1)[0]

print('During NDJFM from '+str(year_start)+ ' to '+ str(year_end)+ 
      ', there are '+ str(len(idx_MJO)) +' MJO days') #(' +str(100*len(idx_MJO)/total_days_ndjfm)+' %)')
days_MJO = [np.datetime64(datetime.datetime(year[i],month[i],day[i])) for i in idx_MJO]
#print(days_MJO)

idx_nMJO = np.where(np.bitwise_not(cond_year & cond_ndjfm & cond_phase24 & cond_amplitude1))[0]
#print(len(idx_nMJO))
days_nMJO = [np.datetime64(datetime.datetime(year[i],month[i],day[i])) for i in idx_nMJO]

In [ ]:
# Investigate impacts of both CS and MJO events

# CS+MJO - Days in NDJFM with CSs and MJO events occured
# CS+nMJO - Days in NDJFM with only CSs and no MJO as defined above.
# MJO+nCS - Days in NDJFM with only MJO events.

days_CSMJO = days_CS.where((days_CS.isin(days_MJO)), drop=True)
#print(len(days_CSMJO))
days_CSnMJO = days_CS.where(np.bitwise_not((days_CS.isin(days_MJO))), drop=True)
#print(len(days_CSnMJO))
days_nCSMJO = days_nCS.where((days_nCS.isin(days_MJO)), drop=True)
#print(len(days_nCSMJO))

In [ ]:
# Investigate the impact on precipitation anomaly from interaction CS and MJO

## precip from CMORPH Dataset (0.25 deg)

#Southeast Asia Domain
min_lon = 90 #89.26
max_lon = 130 #146.96
min_lat = -10 #-15.14
max_lat = 30 #27.26
lat_bnd = [min_lat,max_lat]
lon_bnd = [min_lon,max_lon]

dsp = xr.open_mfdataset('data/CMORPH_SEA*.nc')
dsp_sea = dsp.sel(lat=slice(*lat_bnd),lon=slice(*lon_bnd),time=slice(*time_bnd))
dsp_sea_ndjfm = dsp_sea.where(((dsp_sea['time.month'].isin([11,12,1,2,3]))), drop=True)
# mean prec
mean_prec_ndjfm = dsp_sea_ndjfm['cmorph'].mean(dim='time')

# mean prec during CS days
cs_prec_ndjfm = dsp_sea.where(((dsp_sea['time'].isin(days_CS))), drop=True)
mean_cs_prec_ndjfm = cs_prec_ndjfm['cmorph'].mean(dim='time')
# anomaly
anom_cs_prec_ndjfm = mean_cs_prec_ndjfm - mean_prec_ndjfm

# mean prec during MJO days
mjo_prec_ndjfm = dsp_sea.where(((dsp_sea['time'].isin(days_MJO))), drop=True)
mean_mjo_prec_ndjfm = mjo_prec_ndjfm['cmorph'].mean(dim='time')
# anomaly
anom_mjo_prec_ndjfm = mean_mjo_prec_ndjfm - mean_prec_ndjfm

# mean prec during CS+MJO days
csmjo_prec_ndjfm = dsp_sea.where(((dsp_sea['time'].isin(days_CSMJO))), drop=True)
mean_csmjo_prec_ndjfm = csmjo_prec_ndjfm['cmorph'].mean(dim='time')
# anomaly
anom_csmjo_prec_ndjfm = mean_csmjo_prec_ndjfm - mean_prec_ndjfm

In [ ]:
dsu = xr.open_mfdataset('data/u850_*.nc')
dsu_sea = dsu.sel(lat=slice(*lat_bnd),lon=slice(*lon_bnd),time=slice(*time_bnd))
dsu_sea_ndjfm = dsu_sea.where(((dsu_sea['time.month'].isin([11,12,1,2,3]))), drop=True)
# mean u
mean_u_ndjfm = dsu_sea_ndjfm['u'].mean(dim='time')

# mean u during CS days
cs_u_ndjfm = dsu_sea.where(((dsu_sea['time'].isin(days_CS))), drop=True)
mean_cs_u_ndjfm = cs_u_ndjfm['u'].mean(dim='time')
# anomaly 
anom_cs_u_ndjfm = mean_cs_u_ndjfm - mean_u_ndjfm

# mean u during MJO days
mjo_u_ndjfm = dsu_sea.where(((dsu_sea['time'].isin(days_MJO))), drop=True)
mean_mjo_u_ndjfm = mjo_u_ndjfm['u'].mean(dim='time')
# anomaly
anom_mjo_u_ndjfm = mean_mjo_u_ndjfm - mean_u_ndjfm

# mean u during CS+MJO days
csmjo_u_ndjfm = dsu_sea.where(((dsu_sea['time'].isin(days_CSMJO))), drop=True)
mean_csmjo_u_ndjfm = csmjo_u_ndjfm['u'].mean(dim='time')
# anomaly
anom_csmjo_u_ndjfm = mean_csmjo_u_ndjfm - mean_u_ndjfm

In [ ]:
dsv = xr.open_mfdataset('data/v850_*.nc')
dsv_sea = dsv.sel(lat=slice(*lat_bnd),lon=slice(*lon_bnd),time=slice(*time_bnd))
dsv_sea_ndjfm = dsv_sea.where(((dsv_sea['time.month'].isin([11,12,1,2,3]))), drop=True)
# mean v 
mean_v_ndjfm = dsv_sea_ndjfm['v'].mean(dim='time')

# mean v during CS days
cs_v_ndjfm = dsv_sea.where(((dsv_sea['time'].isin(days_CS))), drop=True)
mean_cs_v_ndjfm = cs_v_ndjfm['v'].mean(dim='time')
# anomaly 
anom_cs_v_ndjfm = mean_cs_v_ndjfm - mean_v_ndjfm

# mean v during MJO days
mjo_v_ndjfm = dsv_sea.where(((dsv_sea['time'].isin(days_MJO))), drop=True)
mean_mjo_v_ndjfm = mjo_v_ndjfm['v'].mean(dim='time')
# anomaly
anom_mjo_v_ndjfm = mean_mjo_v_ndjfm - mean_v_ndjfm

# mean v during CS+MJO days
csmjo_v_ndjfm = dsv_sea.where(((dsv_sea['time'].isin(days_CSMJO))), drop=True)
mean_csmjo_v_ndjfm = csmjo_v_ndjfm['v'].mean(dim='time')
# anomaly
anom_csmjo_v_ndjfm = mean_csmjo_v_ndjfm - mean_v_ndjfm

In [ ]:
# calculate wind speed and wind direction
[ws,wd] = windcalc(mean_u_ndjfm.values, mean_v_ndjfm.values)
mean_ws_ndjfm = mean_u_ndjfm.copy()
# mean ws
mean_ws_ndjfm.values = ws

[ws,wd] = windcalc(mean_cs_u_ndjfm.values, mean_cs_v_ndjfm.values)
mean_cs_ws_ndjfm = mean_cs_u_ndjfm.copy()
# mean ws during CS days
mean_cs_ws_ndjfm.values = ws
# anomaly 
anom_cs_ws_ndjfm = mean_cs_ws_ndjfm - mean_ws_ndjfm

[ws,wd] = windcalc(mean_mjo_u_ndjfm.values, mean_mjo_v_ndjfm.values)
mean_mjo_ws_ndjfm = mean_mjo_u_ndjfm.copy()
# mean ws during MJO days
mean_mjo_ws_ndjfm.values = ws
# anomaly 
anom_mjo_ws_ndjfm = mean_mjo_ws_ndjfm - mean_ws_ndjfm

[ws,wd] = windcalc(mean_csmjo_u_ndjfm.values, mean_csmjo_v_ndjfm.values)
mean_csmjo_ws_ndjfm = mean_csmjo_u_ndjfm.copy()
# mean ws during CS+MJO days
mean_csmjo_ws_ndjfm.values = ws
# anomaly 
anom_csmjo_ws_ndjfm = mean_csmjo_ws_ndjfm - mean_ws_ndjfm

In [ ]:
## Plotting part - CS days
pl.rcParams.update({'font.size': 9})
fig, axs = pl.subplots(nrows=3,ncols=2, gridspec_kw={'hspace': 0.05, 'wspace': 0.05},
                        subplot_kw={'projection': ccrs.PlateCarree()},
                        figsize=(10, 20))
fig.tight_layout()
#pl.tight_layout()
axs=axs.flatten()

levels = np.arange(0,24)
cs=axs[0].contourf(mean_prec_ndjfm.lon,mean_prec_ndjfm.lat,
                   mean_prec_ndjfm, transform = ccrs.PlateCarree(),cmap='terrain_r',extend='max',levels=levels)
axs[0].coastlines()
axs[0].set_title('Mean Precip. NDJFM')
gl = axs[0].gridlines(draw_labels=True)
gl.xlabels_top = False
gl.ylabels_right= False
gl.xlines = False
gl.ylines = False
cbar = pl.colorbar(cs, ax=axs[0], orientation='horizontal')
#cbar.set_ticks(np.linspace(1,10,19))
cbar.set_label('precip [mm/day]')

levels = np.arange(0,17)
cs=axs[1].contourf(mean_ws_ndjfm.lon,mean_ws_ndjfm.lat,
                   mean_ws_ndjfm, transform = ccrs.PlateCarree(),cmap='jet',extend='max',levels=levels)
skip = (slice(None, None, 5), slice(None, None, 5))
cs1=axs[1].quiver(mean_ws_ndjfm.lon.values[slice(None, None, 5)],mean_ws_ndjfm.lat.values[slice(None, None, 5)],
                   mean_u_ndjfm[skip], mean_v_ndjfm[skip], color='white',scale=100,transform=ccrs.PlateCarree())
axs[1].coastlines()
axs[1].set_title('Mean Wind Speed NDJFM')
gl = axs[1].gridlines(draw_labels=True)
gl.xlabels_top = False
gl.ylabels_right= False
gl.xlines = False
gl.ylines = False
cbar = fig.colorbar(cs, ax=axs[1],orientation='horizontal')
#cbar.set_ticks(np.linspace(1,10,16))
cbar.set_label('wind speed [m/s]')

levels = np.arange(0,24)
cs=axs[2].contourf(mean_cs_prec_ndjfm.lon,mean_cs_prec_ndjfm.lat,
                   mean_cs_prec_ndjfm, transform = ccrs.PlateCarree(),cmap='terrain_r',extend='max',levels=levels)
axs[2].coastlines()
axs[2].set_title('Mean Precip. during CS days')
gl = axs[2].gridlines(draw_labels=True)
gl.xlabels_top = False
gl.ylabels_right= False
gl.xlines = False
gl.ylines = False
cbar = fig.colorbar(cs, ax=axs[2],orientation='horizontal')
#cbar.set_ticks(np.linspace(1,10,19))
cbar.set_label('precip [mm/day]')

levels = np.arange(0,17)
cs=axs[3].contourf(mean_cs_ws_ndjfm.lon,mean_cs_ws_ndjfm.lat,
                   mean_cs_ws_ndjfm, transform = ccrs.PlateCarree(),cmap='jet',extend='max',levels=levels)
skip = (slice(None, None, 5), slice(None, None, 5))
cs1=axs[3].quiver(mean_cs_ws_ndjfm.lon.values[slice(None, None, 5)],mean_cs_ws_ndjfm.lat.values[slice(None, None, 5)],
                   mean_cs_u_ndjfm[skip], mean_cs_v_ndjfm[skip], color='white',scale=100,transform=ccrs.PlateCarree())
axs[3].coastlines()
axs[3].set_title('Mean Wind Speed during CS days')
gl = axs[3].gridlines(draw_labels=True)
gl.xlabels_top = False
gl.ylabels_right= False
gl.xlines = False
gl.ylines = False
cbar = fig.colorbar(cs, ax=axs[3],orientation='horizontal')
#cbar.set_ticks(np.linspace(1,10,16))
cbar.set_label('wind speed [m/s]')

levels = np.arange(-12,13)
cs=axs[4].contourf(anom_cs_prec_ndjfm.lon,anom_cs_prec_ndjfm.lat,
                   anom_cs_prec_ndjfm, transform = ccrs.PlateCarree(),cmap='BrBG',extend='both',levels=levels)
axs[4].coastlines()
axs[4].set_title('Anomaly Precip. during CS days')
gl = axs[4].gridlines(draw_labels=True)
gl.xlabels_top = False
gl.ylabels_right= False
gl.xlines = False
gl.ylines = False
cbar = fig.colorbar(cs, ax=axs[4],orientation='horizontal')
#cbar.set_ticks(np.linspace(1,10,19))
cbar.set_label('precip [mm/day]')

levels = np.arange(-6,7)
cs=axs[5].contourf(anom_cs_ws_ndjfm.lon,anom_cs_ws_ndjfm.lat,
                   anom_cs_ws_ndjfm, transform = ccrs.PlateCarree(),cmap='bwr',extend='both',levels=levels)
skip = (slice(None, None, 5), slice(None, None, 5))
cs1=axs[5].quiver(anom_cs_ws_ndjfm.lon.values[slice(None, None, 5)],anom_cs_ws_ndjfm.lat.values[slice(None, None, 5)],
                   anom_cs_u_ndjfm[skip], anom_cs_v_ndjfm[skip], color='black',scale=100,transform=ccrs.PlateCarree())
axs[5].coastlines()
axs[5].set_title('Anomaly Wind Speed during CS days')
gl = axs[5].gridlines(draw_labels=True)
gl.xlabels_top = False
gl.ylabels_right= False
gl.xlines = False
gl.ylines = False
cbar = fig.colorbar(cs, ax=axs[5],orientation='horizontal')
#cbar.set_ticks(np.linspace(1,10,16))
cbar.set_label('wind speed [m/s]')

# save figure
fig.savefig('CS.png', format='png', dpi=150, bbox_inches='tight')

In [ ]:
## Plotting part - MJO days
pl.rcParams.update({'font.size': 9})
fig, axs = pl.subplots(nrows=3,ncols=2, gridspec_kw={'hspace': 0.05, 'wspace': 0.05},
                        subplot_kw={'projection': ccrs.PlateCarree()},
                        figsize=(10, 20))
fig.tight_layout()
#pl.tight_layout()
axs=axs.flatten()

levels = np.arange(0,24)
cs=axs[0].contourf(mean_prec_ndjfm.lon,mean_prec_ndjfm.lat,
                   mean_prec_ndjfm, transform = ccrs.PlateCarree(),cmap='terrain_r',extend='max',levels=levels)
axs[0].coastlines()
axs[0].set_title('Mean Precip. NDJFM')
gl = axs[0].gridlines(draw_labels=True)
gl.xlabels_top = False
gl.ylabels_right= False
gl.xlines = False
gl.ylines = False
cbar = fig.colorbar(cs, ax=axs[0],orientation='horizontal')
#cbar.set_ticks(np.linspace(1,10,19))
cbar.set_label('precip [mm/day]')

levels = np.arange(0,17)
cs=axs[1].contourf(mean_ws_ndjfm.lon,mean_ws_ndjfm.lat,
                   mean_ws_ndjfm, transform = ccrs.PlateCarree(),cmap='jet',extend='max',levels=levels)
skip = (slice(None, None, 5), slice(None, None, 5))
cs1=axs[1].quiver(mean_ws_ndjfm.lon.values[slice(None, None, 5)],mean_ws_ndjfm.lat.values[slice(None, None, 5)],
                   mean_u_ndjfm[skip], mean_v_ndjfm[skip], color='white',scale=100,transform=ccrs.PlateCarree())
axs[1].coastlines()
axs[1].set_title('Mean Wind Speed NDJFM')
gl = axs[1].gridlines(draw_labels=True)
gl.xlabels_top = False
gl.ylabels_right= False
gl.xlines = False
gl.ylines = False
cbar = fig.colorbar(cs, ax=axs[1],orientation='horizontal')
#cbar.set_ticks(np.linspace(1,10,16))
cbar.set_label('wind speed [m/s]')

levels = np.arange(0,24)
cs=axs[2].contourf(mean_mjo_prec_ndjfm.lon,mean_mjo_prec_ndjfm.lat,
                   mean_mjo_prec_ndjfm, transform = ccrs.PlateCarree(),cmap='terrain_r',extend='max',levels=levels)
axs[2].coastlines()
axs[2].set_title('Mean Precip. during MJO days')
gl = axs[2].gridlines(draw_labels=True)
gl.xlabels_top = False
gl.ylabels_right= False
gl.xlines = False
gl.ylines = False
cbar = fig.colorbar(cs, ax=axs[2],orientation='horizontal')
#cbar.set_ticks(np.linspace(1,10,19))
cbar.set_label('precip [mm/day]')

levels = np.arange(0,17)
cs=axs[3].contourf(mean_mjo_ws_ndjfm.lon,mean_mjo_ws_ndjfm.lat,
                   mean_mjo_ws_ndjfm, transform = ccrs.PlateCarree(),cmap='jet',extend='max',levels=levels)
skip = (slice(None, None, 5), slice(None, None, 5))
cs1=axs[3].quiver(mean_mjo_ws_ndjfm.lon.values[slice(None, None, 5)],mean_mjo_ws_ndjfm.lat.values[slice(None, None, 5)],
                   mean_mjo_u_ndjfm[skip], mean_mjo_v_ndjfm[skip], color='white',scale=100,transform=ccrs.PlateCarree())
axs[3].coastlines()
axs[3].set_title('Mean Wind Speed during MJO days')
gl = axs[3].gridlines(draw_labels=True)
gl.xlabels_top = False
gl.ylabels_right= False
gl.xlines = False
gl.ylines = False
cbar = fig.colorbar(cs, ax=axs[3],orientation='horizontal')
#cbar.set_ticks(np.linspace(1,10,16))
cbar.set_label('wind speed [m/s]')

levels = np.arange(-12,13)
cs=axs[4].contourf(anom_mjo_prec_ndjfm.lon,anom_mjo_prec_ndjfm.lat,
                   anom_mjo_prec_ndjfm, transform = ccrs.PlateCarree(),cmap='BrBG',extend='both',levels=levels)
axs[4].coastlines()
axs[4].set_title('Anomaly Precip. during MJO days')
gl = axs[4].gridlines(draw_labels=True)
gl.xlabels_top = False
gl.ylabels_right= False
gl.xlines = False
gl.ylines = False
cbar = fig.colorbar(cs, ax=axs[4],orientation='horizontal')
#cbar.set_ticks(np.linspace(1,10,19))
cbar.set_label('precip [mm/day]')

levels = np.arange(-6,7)
cs=axs[5].contourf(anom_mjo_ws_ndjfm.lon,anom_mjo_ws_ndjfm.lat,
                   anom_mjo_ws_ndjfm, transform = ccrs.PlateCarree(),cmap='bwr',extend='both',levels=levels)
skip = (slice(None, None, 5), slice(None, None, 5))
cs1=axs[5].quiver(anom_mjo_ws_ndjfm.lon.values[slice(None, None, 5)],anom_mjo_ws_ndjfm.lat.values[slice(None, None, 5)],
                   anom_mjo_u_ndjfm[skip], anom_mjo_v_ndjfm[skip], color='black',scale=100,transform=ccrs.PlateCarree())
axs[5].coastlines()
axs[5].set_title('Anomaly Wind Speed during MJO days')
gl = axs[5].gridlines(draw_labels=True)
gl.xlabels_top = False
gl.ylabels_right= False
gl.xlines = False
gl.ylines = False
cbar = fig.colorbar(cs, ax=axs[5],orientation='horizontal')
#cbar.set_ticks(np.linspace(1,10,16))
cbar.set_label('wind speed [m/s]')

# save figure
fig.savefig('MJO.png', format='png', dpi=150, bbox_inches='tight')

In [ ]:
## Plotting part - CS + MJO days
pl.rcParams.update({'font.size': 9})
fig, axs = pl.subplots(nrows=3,ncols=2, gridspec_kw={'hspace': 0.05, 'wspace': 0.05},
                        subplot_kw={'projection': ccrs.PlateCarree()},
                        figsize=(10, 20))
fig.tight_layout()
#pl.tight_layout()
axs=axs.flatten()

levels = np.arange(0,24)
cs=axs[0].contourf(mean_prec_ndjfm.lon,mean_prec_ndjfm.lat,
                   mean_prec_ndjfm, transform = ccrs.PlateCarree(),cmap='terrain_r',extend='max',levels=levels)
axs[0].coastlines()
axs[0].set_title('Mean Precip. NDJFM')
gl = axs[0].gridlines(draw_labels=True)
gl.xlabels_top = False
gl.ylabels_right= False
gl.xlines = False
gl.ylines = False
cbar = fig.colorbar(cs, ax=axs[0],orientation='horizontal')
#cbar.set_ticks(np.linspace(1,10,19))
cbar.set_label('precip [mm/day]')

levels = np.arange(0,17)
cs=axs[1].contourf(mean_ws_ndjfm.lon,mean_ws_ndjfm.lat,
                   mean_ws_ndjfm, transform = ccrs.PlateCarree(),cmap='jet',extend='max',levels=levels)
skip = (slice(None, None, 5), slice(None, None, 5))
cs1=axs[1].quiver(mean_ws_ndjfm.lon.values[slice(None, None, 5)],mean_ws_ndjfm.lat.values[slice(None, None, 5)],
                   mean_u_ndjfm[skip], mean_v_ndjfm[skip], color='white',scale=100,transform=ccrs.PlateCarree())
axs[1].coastlines()
axs[1].set_title('Mean Wind Speed NDJFM')
gl = axs[1].gridlines(draw_labels=True)
gl.xlabels_top = False
gl.ylabels_right= False
gl.xlines = False
gl.ylines = False
cbar = fig.colorbar(cs, ax=axs[1],orientation='horizontal')
#cbar.set_ticks(np.linspace(1,10,16))
cbar.set_label('wind speed [m/s]')

levels = np.arange(0,24)
cs=axs[2].contourf(mean_csmjo_prec_ndjfm.lon,mean_csmjo_prec_ndjfm.lat,
                   mean_csmjo_prec_ndjfm, transform = ccrs.PlateCarree(),cmap='terrain_r',extend='max',levels=levels)
axs[2].coastlines()
axs[2].set_title('Mean Precip. during CS+MJO days')
gl = axs[2].gridlines(draw_labels=True)
gl.xlabels_top = False
gl.ylabels_right= False
gl.xlines = False
gl.ylines = False
cbar = fig.colorbar(cs, ax=axs[2],orientation='horizontal')
#cbar.set_ticks(np.linspace(1,10,19))
cbar.set_label('precip [mm/day]')

levels = np.arange(0,17)
cs=axs[3].contourf(mean_csmjo_ws_ndjfm.lon,mean_csmjo_ws_ndjfm.lat,
                   mean_csmjo_ws_ndjfm, transform = ccrs.PlateCarree(),cmap='jet',extend='max',levels=levels)
skip = (slice(None, None, 5), slice(None, None, 5))
cs1=axs[3].quiver(mean_csmjo_ws_ndjfm.lon.values[slice(None, None, 5)],mean_csmjo_ws_ndjfm.lat.values[slice(None, None, 5)],
                   mean_csmjo_u_ndjfm[skip], mean_csmjo_v_ndjfm[skip], color='white',scale=100,transform=ccrs.PlateCarree())
axs[3].coastlines()
axs[3].set_title('Mean Wind Speed during CS+MJO days')
gl = axs[3].gridlines(draw_labels=True)
gl.xlabels_top = False
gl.ylabels_right= False
gl.xlines = False
gl.ylines = False
cbar = fig.colorbar(cs, ax=axs[3],orientation='horizontal')
#cbar.set_ticks(np.linspace(1,10,16))
cbar.set_label('wind speed [m/s]')

levels = np.arange(-12,13)
cs=axs[4].contourf(anom_csmjo_prec_ndjfm.lon,anom_csmjo_prec_ndjfm.lat,
                   anom_csmjo_prec_ndjfm, transform = ccrs.PlateCarree(),cmap='BrBG',extend='both',levels=levels)
axs[4].coastlines()
axs[4].set_title('Anomaly Precip. during CS+MJO days')
gl = axs[4].gridlines(draw_labels=True)
gl.xlabels_top = False
gl.ylabels_right= False
gl.xlines = False
gl.ylines = False
cbar = fig.colorbar(cs, ax=axs[4],orientation='horizontal')
#cbar.set_ticks(np.linspace(1,10,19))
cbar.set_label('precip [mm/day]')

levels = np.arange(-6,7)
cs=axs[5].contourf(anom_csmjo_ws_ndjfm.lon,anom_csmjo_ws_ndjfm.lat,
                   anom_csmjo_ws_ndjfm, transform = ccrs.PlateCarree(),cmap='bwr',extend='both',levels=levels)
skip = (slice(None, None, 5), slice(None, None, 5))
cs1=axs[5].quiver(anom_csmjo_ws_ndjfm.lon.values[slice(None, None, 5)],anom_csmjo_ws_ndjfm.lat.values[slice(None, None, 5)],
                   anom_csmjo_u_ndjfm[skip], anom_csmjo_v_ndjfm[skip], color='black',scale=100,transform=ccrs.PlateCarree())
axs[5].coastlines()
axs[5].set_title('Anomaly Wind Speed during CS+MJO days')
gl = axs[5].gridlines(draw_labels=True)
gl.xlabels_top = False
gl.ylabels_right= False
gl.xlines = False
gl.ylines = False
cbar = fig.colorbar(cs, ax=axs[5],orientation='horizontal')
#cbar.set_ticks(np.linspace(1,10,16))
cbar.set_label('wind speed [m/s]')

# save figure
fig.savefig('CSMJO.png', format='png', dpi=150, bbox_inches='tight')

### More references :

Xavier, P., Lim, S.Y., Abdullah, M.F.A.B., Bala, M., Chenoli, S.N., 
Handayani, A.S., Marzin, C., Permana, D., Tangang, F., Williams, K.D. 
and Yik, D.J., 2020. Seasonal dependence of cold surges and their 
interaction with the madden–julian oscillation over Southeast Asia. 
Journal of Climate, 33(6), pp.2467-2482.

Diong, J. Y., Xavier, P., Woolnough, S. J., & Abdullah, F. A. (2023). 
Equatorial Rossby waves on cold surge days and their impact on rainfall. 
Quarterly Journal of the Royal Meteorological Society, 149(754), 2031-2047.

Lubis, S. W., Hagos, S., Chang, C. C., Balaguru, K., & Leung, L. R. (2023). 
Cross‐Equatorial Surges Boost MJO's Southward Detour Over the Maritime Continent. 
Geophysical Research Letters, 50(15), e2023GL104770.

#### Need further thought about the impact on extreme precipitation from the following interactions  
##### Interaction between CENS (Hattori et al.,2011) with MJO (see Xavier et al., 2020 ; Lubis et al., 2023)
##### Interaction between CS with Eq.Rossby waves (see Diong et al., 2023)
##### Interaction between CS with Eq.Kevin waves (??)
##### Interaction between CS with MRG waves (??)

### Thank you for attending the session !!